In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/gianni/Files/Studium/Master/04_FS2023/MT/f1tenth_development_gym'

In [2]:
from SI_Toolkit.Functions.General.Initialization import get_net, get_norm_info_for_net
from SI_Toolkit.Functions.General.Normalising import get_normalization_function
from SI_Toolkit.computation_library import TensorFlowLibrary
import types
import numpy as np

2023-10-29 23:56:53.842038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 23:56:53.884709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 23:56:53.885350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 23:56:54.666816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-29 23:57:02.987686: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
def load_network(folder, model, model_subfolder='', path=f'./SI_Toolkit_ASF/Experiments'):
    full_path = f'{path}/{folder}/Models/{model_subfolder}/'

    a = types.SimpleNamespace()
    a.net_name = model
    a.path_to_models = full_path
    a.wash_out_len = 0
    a.post_wash_out_len = 1

    net, net_info = get_net(a, batch_size=1)
    return (net, net_info)

def normalize_input(net_input, net_info):
    norm_info = get_norm_info_for_net(net_info)
    norm_func = get_normalization_function(norm_info, net_info.inputs, TensorFlowLibrary)
    return norm_func(net_input)

In [4]:
net, net_info = load_network('0_AL', 'Custom-11IN-ODE_module-STModelMu-9OUT-0')
net.car_parameters_tf

Loading a pretrained network with the full name: Custom-11IN-ODE_module-STModelMu-9OUT-0

Inputs to the loaded network: angular_control, translational_control, angular_vel_z, linear_vel_x, pose_theta, pose_theta_cos, pose_theta_sin, pose_x, pose_y, slip_angle, steering_angle
Outputs from the loaded network: angular_vel_z, linear_vel_x, pose_theta, pose_theta_cos, pose_theta_sin, pose_x, pose_y, slip_angle, steering_angle

Warning! Net info does not contain information about dt!
Loaded the model STModelMu from ./SI_Toolkit_ASF/Modules/ODE_module.py.
Loading Model:  ./SI_Toolkit_ASF/Experiments/0_AL/Models//Custom-11IN-ODE_module-STModelMu-9OUT-0/ckpt.ckpt

Model loaded from a checkpoint.


{'mu': <tf.Variable 'mu:0' shape=() dtype=float32, numpy=0.9>,
 'C_Sf': <tf.Variable 'C_Sf:0' shape=() dtype=float32, numpy=4.718>,
 'C_Sr': <tf.Variable 'C_Sr:0' shape=() dtype=float32, numpy=5.4562>,
 'lf': <tf.Variable 'lf:0' shape=() dtype=float32, numpy=0.15875>,
 'lr': <tf.Variable 'lr:0' shape=() dtype=float32, numpy=0.17145>,
 'h': <tf.Variable 'h:0' shape=() dtype=float32, numpy=0.074>,
 'm': <tf.Variable 'm:0' shape=() dtype=float32, numpy=4.2>,
 'I': <tf.Variable 'I:0' shape=() dtype=float32, numpy=0.04712>,
 'g': <tf.Variable 'g:0' shape=() dtype=float32, numpy=9.81>,
 'width': <tf.Variable 'width:0' shape=() dtype=float32, numpy=0.31>,
 'length': <tf.Variable 'length:0' shape=() dtype=float32, numpy=0.58>,
 's_min': <tf.Variable 's_min:0' shape=() dtype=float32, numpy=-0.4189>,
 's_max': <tf.Variable 's_max:0' shape=() dtype=float32, numpy=0.4189>,
 'sv_min': <tf.Variable 'sv_min:0' shape=() dtype=float32, numpy=-3.2>,
 'sv_max': <tf.Variable 'sv_max:0' shape=() dtype=floa

In [5]:
folder = '0_DNN'
model = 'Dense-10IN-64H1-128H2-64H3-8OUT-0'

net, net_info = load_network(folder, model)
first_input = np.array([[[0.403, 5.89, -5.052, 5.896, 1.0, 0.0, 0.0, 0.0, 0.279, -0.244]]])
first_input = normalize_input(first_input, net_info)
print(net.predict(first_input))

second_input = np.array([[[0.403, 5.89, -5.052, 5.896, 1.0, 0.0, 10000.0, 10000.0, 0.279, -0.244]]])
second_input = normalize_input(second_input, net_info)
print(net.predict(second_input))

Loading a pretrained network with the full name: Dense-10IN-64H1-128H2-64H3-8OUT-0

Inputs to the loaded network: angular_control, translational_control, angular_vel_z, linear_vel_x, pose_theta_cos, pose_theta_sin, pose_x, pose_y, slip_angle, steering_angle
Outputs from the loaded network: D_angular_vel_z, D_linear_vel_x, D_pose_theta_cos, D_pose_theta_sin, D_pose_x, D_pose_y, D_slip_angle, D_steering_angle

Constructed a neural network of type Dense, with 3 hidden layers with sizes 64, 128, 64 respectively.
Loading Model:  ./SI_Toolkit_ASF/Experiments/0_DNN/Models/Dense-10IN-64H1-128H2-64H3-8OUT-0/ckpt.ckpt

Model loaded from a checkpoint.
1/1 [==============================] - 0s 151ms/step
[[[ 0.66690063 -0.02496634  0.07899188 -0.5755585   0.5859602
    0.13937137  0.506636    0.9971581 ]]]
1/1 [==============================] - 0s 16ms/step
[[[ 0.41962606 -0.529468    0.22278497 -0.17148323  0.06865334
   -0.17035156 -0.07038237  1.0255618 ]]]


In [6]:
folder = '0_DNN'
model = 'Dense-8IN-64H1-128H2-64H3-8OUT-0'

net, net_info = load_network(folder, model)
first_input = np.array([[[0.403, 5.89, -5.052, 5.896, 1.0, 0.0, 0.279, -0.244]]])
first_input = normalize_input(first_input, net_info)
print(net.predict(first_input))

Loading a pretrained network with the full name: Dense-8IN-64H1-128H2-64H3-8OUT-0

Inputs to the loaded network: angular_control, translational_control, angular_vel_z, linear_vel_x, pose_theta_cos, pose_theta_sin, slip_angle, steering_angle
Outputs from the loaded network: D_angular_vel_z, D_linear_vel_x, D_pose_theta_cos, D_pose_theta_sin, D_pose_x, D_pose_y, D_slip_angle, D_steering_angle

Constructed a neural network of type Dense, with 3 hidden layers with sizes 64, 128, 64 respectively.
Loading Model:  ./SI_Toolkit_ASF/Experiments/0_DNN/Models/Dense-8IN-64H1-128H2-64H3-8OUT-0/ckpt.ckpt

Model loaded from a checkpoint.
1/1 [==============================] - 0s 106ms/step
[[[ 0.66830236 -0.01968992  0.07744277 -0.57960576  0.58169574
    0.13667008  0.5022316   0.99827665]]]
